In [65]:
# Libraries

import whisper
from datetime import timedelta
import os

In [71]:
# speech

model = whisper.load_model("medium.en",device='cpu')
result = model.transcribe("data/test.mp4")

with open("data/test.txt", "w+") as f:
    f.write(result["text"])

# subtitles

segments = result['segments']

for segment in segments:
    startTime = str(0)+str(timedelta(seconds=int(segment['start'])))+',000'
    endTime = str(0)+str(timedelta(seconds=int(segment['end'])))+',000'
    text = segment['text']
    segmentId = segment['id']+1
    segment = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] == ' ' else text}\n\n"

    srtFilename = os.path.join(r"data", "test.srt")
    with open(srtFilename, 'a', encoding='utf-8') as srtFile:
        srtFile.write(segment)



100%|█████████████████████████████████████| 1.42G/1.42G [06:43<00:00, 3.79MiB/s]
/home/roque/01. IronHack/00. Data Analytics/01. Course/63. Week 23 - Day 3/git/final-project-bootcamp/.venv/lib/python3.9/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [105]:
result['text']

" And I will share my screen. So basically one of the most asked questions for this next six months are can you all see my screen and can you hear me? In case you don't let me know or just write in the chat. Otherwise I just start. OK. So the agenda for my presentation is what do I need to start the course, right? Then what should I expect in the following days? What should I do to successfully complete the bootcamp? And of course, we all want you to complete the bootcamp successful. Will be my point of contact and then you will meet your teaching team. So what do I need to start the course? Right. First, very important. Be on Slack. I know if you are working with Slack for the first time, it can be a little bit confusing because we are working with two different workspaces. You have the general workspace, which is the light blue one. And there you can find every Iron Hacker ever. And also the data pre-work help channel. And then you already received the link for the class workspace. I

In [107]:
result['segments'][1]

{'id': 1,
 'seek': 0,
 'start': 4.0,
 'end': 14.0,
 'text': ' So basically one of the most asked questions for this next six months are can you all see my screen and can you hear me?',
 'tokens': [1406,
  6209,
  530,
  286,
  262,
  749,
  1965,
  2683,
  329,
  428,
  1306,
  2237,
  1933,
  389,
  460,
  345,
  477,
  766,
  616,
  3159,
  290,
  460,
  345,
  3285,
  502,
  30],
 'temperature': 0.0,
 'avg_logprob': -0.28894358775654777,
 'compression_ratio': 1.4493670886075949,
 'no_speech_prob': 0.08684176951646805}

In [118]:
# Summary

# python -m spacy download en_core_web_sm #eficency
# python -m spacy download en_core_web_trf #accuracy

import spacy
import en_core_web_trf
#import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
import spacy_transformers
from string import punctuation
from heapq import nlargest

def summarize(text, per):
    #nlp = spacy.load('en_core_web_trf')
    nlp = en_core_web_trf.load()
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary

In [119]:
summarize(result['text'], 0.10)

"The thing we are so strict on pre-work deadline is at first it's you do yourself a favour, because if you don't do pre-work, you won't be able to follow the course because the course is fast paced.During the bootcamp, in terms of dedication, you will need to know that there will be times where you need to work out of outside bootcamp hours.Then in terms of pre-work, so pre-work should be ready by now.And in order to give them a little bit time to work to go through the pre-work, it's mandatory to have it finished by now.But in terms of labs or projects, for example, you will be asked to work also on other days that are not bootcamp hours.On the other side, your teaching team will look through the pre-work and they need to provide some feedback."

In [117]:
summarize(result['text'], 0.10)

"The thing we are so strict on pre-work deadline is at first it's you do yourself a favour, because if you don't do pre-work, you won't be able to follow the course because the course is fast paced.During the bootcamp, in terms of dedication, you will need to know that there will be times where you need to work out of outside bootcamp hours.Then in terms of pre-work, so pre-work should be ready by now.And in order to give them a little bit time to work to go through the pre-work, it's mandatory to have it finished by now.But in terms of labs or projects, for example, you will be asked to work also on other days that are not bootcamp hours.On the other side, your teaching team will look through the pre-work and they need to provide some feedback."